In [11]:
import os
import json
import requests
from functools import reduce
from datetime import datetime as dt
from datetime import timedelta as td
import numpy as np
import pandas as pd

In [12]:
# API keys. 
EIA_API_KEY = 'GbKYer2Pz9JS0UttTjwxahbfN5ZzDKE0ZuyjsJQY'

# URL query.
url = 'https://api.openei.org/utility_rates?version=latest&format=json&api_key={eia_api_key}&limit=1&detail=full&orderby=startdate&direction=desc&approved=true&sector=Residential&eia={eia_id}'

# Directory path. 
dir = 'data/URDB_Rate_Structures'

# EIA IDs for default time-of-use rates from URDB.
eia_ids = {
     'PGE': 14328,
     'SCE': 17609,
     'SDGE': 17609
}

In [13]:
def build_8760_from_12by24s(wkday_12by24, wkend_12by24, start_day=6):
    """
    Construct long-df (8760) from a weekday and weekend 12by24
    Parameters
    ----------
    wkday_12by24 : numpy.ndarray
    wkend_12by24 : numpy.ndarray
    start_day : int
        Start day of 6 (default) equates to a Sunday.
    """

    month_hours = np.array([0, 744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760], int)

    month_index = np.zeros(8760, int)
    for month, hours in enumerate(month_hours):
        month_index[month_hours[month - 1]: hours] = month - 1

    period_8760 = np.zeros(8760, int)
    hour = 0
    day = start_day
    for h in range(8760):
        if day < 5:
            period_8760[h] = wkday_12by24[month_index[h]][hour]
        else:
            period_8760[h] = wkend_12by24[month_index[h]][hour]
        hour += 1
        if hour == 24: hour = 0; day += 1
        if day == 7: day = 0

    return period_8760


In [14]:
rates = []

for utility in eia_ids:

    # Parameters.
    params = {
        'eia_api_key': EIA_API_KEY,
        'eia_id': eia_ids[utility]
    }

    # Request.
    response = requests.get(url.format(**params))

    # Convert to json. 
    json = response.json()

    # Energy rate name.
    rate_name = utility + '_' + str(json['items'][0]['eiaid'])

    # Energy rate structure.
    rate_structure = json['items'][0]['energyratestructure']

    # Energy weekday schedule.
    weekday_schedule = json['items'][0]['energyweekdayschedule']

    # Energy weekend schedule. 
    weekend_schedule = json['items'][0]['energyweekendschedule']

    # Construct 8760 from weekday and weekend schedules. 
    # NOTE: Jan 1, 2022 is a Saturday (start_day = 5).
    period_8760 = build_8760_from_12by24s(weekday_schedule, weekend_schedule, start_day=5)

    # Convert to dataframe.
    df = pd.DataFrame(period_8760, columns=['Period'])

    # Add timestamp.
    timestamp = dt(2022, 1, 1, 0, 0, 0)
    timestamps = [timestamp]

    for i in range(1, 8760):
        timestamp += td(seconds=3600)
        timestamps.append(timestamp)
    
    df['Timestamp'] = timestamps
    
    # Period to rate.
    period_to_rate = {}
    for period, rate in enumerate(rate_structure):
        period_to_rate[period] = rate[0]['rate']

    # Translate from periods to rates.
    df[rate_name] = df['Period'].replace(period_to_rate)

    # Drop unused columns.
    df = df[['Timestamp', rate_name]]

    # Add to list. 
    rates.append(df)

    # Write dataframe to CSV.
    df.to_csv(f'{dir}/{rate_name}.csv', index=False)

# Merge rates on timestamp.
tous = reduce(lambda x, y: x.merge(y, on='Timestamp'), rates)

# Write dataframe to CSV. 
tous.to_csv('data/Aggregated_TOUs.csv', index=False)